In [1]:
import os, json
import pandas as pd
import boto3
import geopandas as gpd

from math import floor
from pystac_client import Client
import planetary_computer as pc
import numpy as np
import rasterio
import requests, pyproj
from pyproj.aoi import AreaOfInterest
from pyproj.database import query_utm_crs_info
from rasterio.merge import merge as rmerge
import rasterio.mask
from rasterio.features import shapes
from shapely.geometry import Polygon, MultiPolygon, box, shape
from shapely.ops import transform, unary_union
import networkx as nx

In [2]:
import warnings
warnings.filterwarnings(action='ignore')

In [3]:
CONNECTIVITY_DISTANCE = 100    # Max distance two patches can be apart and be considered connected (meter)
MIN_PATCHSIZE = 10000    # Min patch size to be included in analysis (sq meter)

In [4]:
# define directory
out_dir = os.getcwd()
bucket_name = 'cities-indicators'
aws_s3_dir = "https://"+bucket_name+".s3.eu-west-3.amazonaws.com"
boundary_ext = '/data/boundaries/'
indicators_file_aws = 'indicators/indicators.csv'

In [15]:
OUTPUT_FILENAME = 'BIO_2_habitat_connectivity.csv'

In [16]:
# get list of cities
boundary_georef = pd.read_csv(aws_s3_dir + boundary_ext + 'boundary_georef.csv')
boundary_georef

,Unnamed: 0,geo_name,level,aoi_boundary_name,units_boundary_name,city_name,country_name,country_code,continent,city_id,project_name,city_boundary_name,geo_level
0,1,ARG-Mendoza,region,ADM3union,ADM3,Mendoza,Argentina,ARG,America,ARG-Mendoza_ADM-3-union_1,urbanshift,NaN,NaN
1,2,ARG-Mar_del_Plata,city,ADM3,ADM4,Mar del Plata city,Argentina,ARG,America,ARG-Mar_del_Plata_ADM-3_1,urbanshift,NaN,NaN
2,3,ARG-Ushuaia,city,ADM4,ADM5,Ushuaia city,Argentina,ARG,America,ARG-Ushuaia_ADM-4_1,urbanshift,NaN,NaN
3,4,ARG-Salta,region,ADM2union,ADM3,Salta,Argentina,ARG,America,ARG-Salta_ADM-2-union_1,urbanshift,NaN,NaN
4,5,ARG-Buenos_Aires,region,ADM2union,ADM2,Buenos Aires,Argentina,ARG,America,ARG-Buenos_Aires_ADM-2-union_1,urbanshift,NaN,NaN
5,6,BRA-Teresina,city,ADM4union,ADM4,Teresina city,Brazil,BRA,America,BRA-Teresina_ADM-4-union_1,urbanshift,NaN,NaN
6,7,BRA-Florianopolis,city,ADM4union,ADM4,Florianopolis,Brazil,BRA,America,BRA-Florianopolis_ADM-4-union_1,urbanshift,NaN,NaN
7,8,BRA-Belem,city,ADM4union,ADM4,Belem city,Brazil,BRA,America,BRA-Belem_ADM-4-union_1,urbanshift,NaN,NaN
8,9,CRI-San_Jose,region,ADM2union,ADM2,San Jose,Costa Rica,CRI,America,CRI-San_Jose_ADM-2-union_1,urbanshift,NaN,NaN
9,10,RWA-Kigali,city,ADM4union,ADM4,Kigali,Rwanda,RWA,Africa,RWA-Kigali_ADM-4-union_1,urbanshift,NaN,NaN


In [8]:
# Convert district-boundary geojsons to Shapely polygons

def geojson_to_polygons(g):
    result = []
    for feature in g['features']:
        name = feature['properties']['geo_name']
        if type(feature['geometry']['coordinates'][0][0][0]) == list:
            coordpairs = [(float(i[0]), float(i[1])) for i in feature['geometry']['coordinates'][0][0]]
        else:
            coordpairs = [(float(i[0]), float(i[1])) for i in feature['geometry']['coordinates'][0]]
        result.append((name, Polygon(coordpairs)))
    return result

#district_polys = geojson_to_polygons(Districts_json)

In [9]:
# This function clips and masks raster
# Adapted from https://gis.stackexchange.com/a/387772

def mask_raster_with_geometry(raster, transform, shapes, **kwargs):
    """Wrapper for rasterio.mask.mask to allow for in-memory processing.

    Docs: https://rasterio.readthedocs.io/en/latest/api/rasterio.mask.html

    Args:
        raster (numpy.ndarray): raster to be masked with dim: [H, W]
        transform (affine.Affine): the transform of the raster
        shapes, **kwargs: passed to rasterio.mask.mask

    Returns:
        masked: numpy.ndarray or numpy.ma.MaskedArray with dim: [H, W], and new affine transform
    """
    with rasterio.io.MemoryFile() as memfile:
        with memfile.open(
            driver='GTiff',
            height=raster.shape[0],
            width=raster.shape[1],
            count=1,
            dtype=raster.dtype,
            transform=transform,
        ) as dataset:
            dataset.write(raster, 1)
        with memfile.open() as dataset:
            output, new_transform = rasterio.mask.mask(dataset, shapes, **kwargs)
    return output.squeeze(0), new_transform

In [10]:
# Reclassify from ESA WorldCover classes to habitat/nonhabitat

def classify_habitat(r):  # Note: order is important
    r[r == 60] = 0    # sparse veg
    r[r >= 90] = 1    # herbaceous wetland, mangrove, lichen & moss
    r[r == 80] = 0    # permanent open water
    r[r == 70] = 0    # snow/ice
    r[r == 40] = 0    # cropland
    r[r == 50] = 0    # built up
    r[r == 30] = 1    # grassland
    r[r == 20] = 1    # shrubland
    r[r == 10] = 1    # tree cover
    r[r == 0] = 0     # no data

    return r

In [11]:
def within_distance(rownum, gdf, sidx):
    if rownum % 1000 == 0:
        print('     {0} / {1}'.format(rownum, gdf.count().geometry))
    z = list(sidx.intersection(gdf.iloc[rownum].geometry.buffer(CONNECTIVITY_DISTANCE).bounds))  # Just look at patches in bounding box of buffered
    distances = gdf.iloc[z].distance(gdf.iloc[rownum].geometry)
    return [i for i in z if distances[i] <= CONNECTIVITY_DISTANCE if i != rownum]  # Return only those in actual buffer

In [12]:
def do_one_geom(row):
    print(row['geo_id'], row['geo_name'])
    geom = row[0]
    if type(geom) == MultiPolygon:
        district_poly = MultiPolygon(row[0])
    else:
        district_poly = MultiPolygon([row[0]])
    bounds = district_poly.bounds
    centroid_lon = district_poly.centroid.xy[0][0]
    centroid_lat = district_poly.centroid.xy[1][0]
    target_epsg = (32600 + [0, 100][int(centroid_lat < 0)]) + floor((180 + centroid_lon) / 6) + 1
    # EPSG is 32600 (or 32700 if lat is neg) + longitude zone. Each zone is six degrees, and first zone is 1.
    # Transform from EPSG:4326 to target EPSG
    project = pyproj.Transformer.from_crs(
        pyproj.CRS.from_epsg(4326), # source coordinate system
        pyproj.CRS.from_epsg(target_epsg), # destination coordinate system
        always_xy=True
    )
    # Get ESA WorldCover raster tiles covering AOI
    catalog = Client.open("https://planetarycomputer.microsoft.com/api/stac/v1")
    search = catalog.search(
        collections=["esa-worldcover"],
        bbox=bounds,
    )
    items = list(search.get_items())
    rasters_to_mosaic = []
    signed_hrefs = [pc.sign(i.assets["map"].href) for i in items]
    for href in signed_hrefs:
        raster = rasterio.open(href)
        rasters_to_mosaic.append(raster)

    # Stitch rasters together
    mosaic, mosaic_transform = rmerge(rasters_to_mosaic)
    # Clip raster to district boundary
    clipped_raster, clip_transform = mask_raster_with_geometry(mosaic[0], mosaic_transform, district_poly, crop=True)
    # Classify clipped raster as habitat/nonhabitat
    hab_raster = classify_habitat(clipped_raster)
    # Vectorize and collect only the habitat patches (as opposed to nonhabitat)
    with rasterio.Env():
        image = hab_raster
        results = (
        {'properties': {'raster_val': v}, 'geometry': s}
        for (s, v) 
        in shapes(image, transform=clip_transform) if v == 1)
    shapelist = list(results)
    all_patches = [  # project to UTM so that spatial unit is meter
        transform(project.transform, shape(j['geometry'])) for j in shapelist
    ]
    patches = [
        i for i in [j.simplify(100, preserve_topology=False) for j in all_patches] if i.area >= MIN_PATCHSIZE # Remove very small patches from consideration
#        j for j in all_patches if j.area >= MIN_PATCHSIZE # Remove very small patches from consideration
    ]
    patchgeoms = gpd.GeoDataFrame(geometry=patches, crs='EPSG:{}'.format(target_epsg), index=range(len(patches)))
    patchgeoms_sindex = patchgeoms.sindex


    connected = {
        j: within_distance(j, patchgeoms, patchgeoms_sindex) for j in range(len(patches))
    }
    # Find clusters from connected pairs
    edges = []
    for k in connected:
        for j in connected[k]:
            edges.append((k, j))
    G = nx.Graph()
    G.add_nodes_from(range(len(patches)))
    G.add_edges_from(edges)
    clusters = nx.connected_components(G)
    # Calculate indicator
    total_area = sum([j.area for j in patches])
    cluster_areas = []
    for k in clusters:
        cluster_areas.append(sum([patches[j].area for j in k]))
    if total_area > 0:
        return sum([j**2 for j in cluster_areas]) / (total_area**2)
    else:
        return 0

In [ ]:
%%time
results = []
# Some cities may have memory challenges. They can be run in batches by adjusting the range number in the following line, producing multiple OUTPUT_FILENAME files, and then combining into one.
for i in range(0,len(boundary_georef)): 
    for boundary_name in ['aoi_boundary_name', 'units_boundary_name']:
        if type(boundary_georef.loc[i, boundary_name]) != float: # sometimes boundary_id is nan
            boundary_id = boundary_georef.loc[i, 'geo_name'] + '-' + boundary_georef.loc[i, boundary_name]
            # read boundaries
            boundary_path = aws_s3_dir + boundary_ext +'boundary-'+boundary_id+'.geojson'
            boundary_geo = requests.get(boundary_path).json()
            gdf = gpd.GeoDataFrame.from_features(boundary_geo)
            gdf['BIO_2_habitat_connectivity'] = gdf.apply(do_one_geom, axis=1)
            results.append(gdf[['geo_id', 'geo_name', 'BIO_2_habitat_connectivity']].copy())
            output = pd.concat(results, axis=0)
            output.to_csv(OUTPUT_FILENAME)

CHN-Chongqing_ADM-1_1 CHN-Chongqing


In [ ]:
processedcities = pd.read_csv(OUTPUT_FILENAME)
processedcities

In [8]:
# csv1 = pd.read_csv('BIO_2_habitat_connectivity.csv')
# csv2 = pd.read_csv('BIO_2_habitat_connectivity2.csv')
# csv3 = pd.read_csv('BIO_2_habitat_connectivity3.csv')

# processedcities = csv1.append(csv3).append(csv2)
# processedcities

,Unnamed: 0,geo_id,geo_name,BIO_2_habitat_connectivity
0,0,ARG-Mendoza_ADM-3-union_1,ARG-Mendoza,0.407088
1,0,ARG-Mendoza_ADM-3_1,Distrito Las Barrancas,0.641266
2,1,ARG-Mendoza_ADM-3_2,Distrito San Roque,0.227415
3,2,ARG-Mendoza_ADM-3_3,Distrito Fray Luis Beltrán,0.387826
4,3,ARG-Mendoza_ADM-3_4,Distrito Rodeo del Medio,0.145720
...,...,...,...,...
1528,82,RWA-Musanze_ADM5_83,Kamajaga,1.000000
1529,83,RWA-Musanze_ADM5_84,Kamicaca,0.889280
1530,84,RWA-Musanze_ADM5_85,Musenyi,1.000000
1531,85,RWA-Musanze_ADM5_86,Rugari,1.000000


# Merge with indicator table

In [9]:
# read indicator table
cities_indicators = pd.read_csv(aws_s3_dir +'/'+ indicators_file_aws)
cities_indicators

,geo_id,geo_level,geo_name,geo_parent_name,HEA_1_percentChangeinDaysAbove35C2020to2050,HEA_4_percentBuiltupWithoutTreeCover,HEA_2_percentBuiltupwHighLST-2013to2022meanofmonthwhottestday,ACC_2_percentOpenSpaceinBuiltup2022,ACC_1_OpenSpaceHectaresper1000people2022,FLD_2_percentChangeinMaxDailyPrecip2020to2050,...,FLD_3_percentBuiltupWithin1mAboveDrainage,FLD_7_percentSteepSlopesWOvegetationcover2020,FLD_6_percentRiparianZonewoVegorWatercover2020,AQ_3_percentWHOlimitPM25exposure2020,GHG_2_meanannualTreeCarbonFluxMgcO2eperHA,BIO_1_percentNaturalArea,LND_1_percentPermeableSurface,LND_2_percentTreeCover,LND_4_percentof2000HabitatAreaRestoredby2020,LND_5_numberofHabitatTypesRestoredby2020
0,ARG-Mendoza_ADM-3-union_1,ADM-3-union,ARG-Mendoza,ARG-Mendoza,0.0,NaN,0.306981,0.037594,350.814444,-0.041587,...,NaN,0.991441,0.834118,395.468104,-0.003951,0.340102,0.982321,NaN,0.011921,1.000000
1,ARG-Mendoza_ADM-3_1,ADM-3,Distrito Las Barrancas,ARG-Mendoza,NaN,NaN,0.317241,0.006540,1.972102,NaN,...,NaN,0.500000,0.642378,421.848765,-0.000560,0.422282,0.998457,NaN,0.150528,0.400000
2,ARG-Mendoza_ADM-3_2,ADM-3,Distrito San Roque,ARG-Mendoza,NaN,NaN,0.005444,0.006642,0.128004,NaN,...,NaN,0.000000,0.448561,418.691308,-0.000316,0.286480,0.976852,NaN,0.167138,0.600000
3,ARG-Mendoza_ADM-3_3,ADM-3,Distrito Fray Luis Beltrán,ARG-Mendoza,NaN,NaN,0.204788,0.007968,0.194904,NaN,...,NaN,0.000000,0.504732,443.270436,-0.001430,0.282508,0.984147,NaN,0.095692,0.833333
4,ARG-Mendoza_ADM-3_4,ADM-3,Distrito Rodeo del Medio,ARG-Mendoza,NaN,NaN,0.126250,0.005204,0.329989,NaN,...,NaN,0.642857,0.399084,435.658329,-0.001232,0.185538,0.964504,NaN,0.101342,0.833333
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2623,RWA-Musanze_ADM5_83,ADM5,Kamajaga,RWA-Musanze,NaN,0.993718,0.078125,0.000000,0.000000,NaN,...,0.000000,0.000000,0.006873,747.936003,-6.014612,0.321897,1.000000,0.233365,0.004286,1.000000
2624,RWA-Musanze_ADM5_84,ADM5,Kamicaca,RWA-Musanze,NaN,0.995015,0.125000,0.000000,0.000000,NaN,...,0.000000,0.000000,0.000000,753.422329,-9.194192,0.536051,1.000000,0.372657,0.047648,1.000000
2625,RWA-Musanze_ADM5_85,ADM5,Musenyi,RWA-Musanze,NaN,0.983103,0.280000,0.000000,0.000000,NaN,...,0.013621,0.000000,0.039106,754.165614,-2.157699,0.214939,1.000000,0.167535,0.000000,0.000000
2626,RWA-Musanze_ADM5_86,ADM5,Rugari,RWA-Musanze,NaN,0.952790,0.000000,0.000000,0.000000,NaN,...,0.001839,0.000000,0.000000,745.213966,-1.362025,0.488671,0.983759,0.256713,0.000000,-9999.000000


In [10]:
def merge_indicators(indicator_table, new_indicator_table, indicator_name):
    if indicator_name in indicator_table.columns:
        print("replace with new calculations")
        indicator_table.drop(indicator_name, inplace=True, axis=1)
        cities_indicators_df = indicator_table.merge(new_indicator_table[["geo_id",indicator_name]], 
                                                     on='geo_id', 
                                                     how='left')
    else:
        print("add new indicators")
        cities_indicators_df = indicator_table.merge(new_indicator_table[["geo_id",indicator_name]], 
                                                     on='geo_id', 
                                                     how='left')
    return(cities_indicators_df)

In [11]:
cities_indicators_merged = merge_indicators(indicator_table = cities_indicators,
                                            new_indicator_table = processedcities,
                                            indicator_name = 'BIO_2_habitat_connectivity')

add new indicators


In [12]:
cities_indicators_merged

,geo_id,geo_level,geo_name,geo_parent_name,HEA_1_percentChangeinDaysAbove35C2020to2050,HEA_4_percentBuiltupWithoutTreeCover,HEA_2_percentBuiltupwHighLST-2013to2022meanofmonthwhottestday,ACC_2_percentOpenSpaceinBuiltup2022,ACC_1_OpenSpaceHectaresper1000people2022,FLD_2_percentChangeinMaxDailyPrecip2020to2050,...,FLD_7_percentSteepSlopesWOvegetationcover2020,FLD_6_percentRiparianZonewoVegorWatercover2020,AQ_3_percentWHOlimitPM25exposure2020,GHG_2_meanannualTreeCarbonFluxMgcO2eperHA,BIO_1_percentNaturalArea,LND_1_percentPermeableSurface,LND_2_percentTreeCover,LND_4_percentof2000HabitatAreaRestoredby2020,LND_5_numberofHabitatTypesRestoredby2020,BIO_2_habitat_connectivity
0,ARG-Mendoza_ADM-3-union_1,ADM-3-union,ARG-Mendoza,ARG-Mendoza,0.0,NaN,0.306981,0.037594,350.814444,-0.041587,...,0.991441,0.834118,395.468104,-0.003951,0.340102,0.982321,NaN,0.011921,1.000000,0.407088
1,ARG-Mendoza_ADM-3_1,ADM-3,Distrito Las Barrancas,ARG-Mendoza,NaN,NaN,0.317241,0.006540,1.972102,NaN,...,0.500000,0.642378,421.848765,-0.000560,0.422282,0.998457,NaN,0.150528,0.400000,0.641266
2,ARG-Mendoza_ADM-3_2,ADM-3,Distrito San Roque,ARG-Mendoza,NaN,NaN,0.005444,0.006642,0.128004,NaN,...,0.000000,0.448561,418.691308,-0.000316,0.286480,0.976852,NaN,0.167138,0.600000,0.227415
3,ARG-Mendoza_ADM-3_3,ADM-3,Distrito Fray Luis Beltrán,ARG-Mendoza,NaN,NaN,0.204788,0.007968,0.194904,NaN,...,0.000000,0.504732,443.270436,-0.001430,0.282508,0.984147,NaN,0.095692,0.833333,0.387826
4,ARG-Mendoza_ADM-3_4,ADM-3,Distrito Rodeo del Medio,ARG-Mendoza,NaN,NaN,0.126250,0.005204,0.329989,NaN,...,0.642857,0.399084,435.658329,-0.001232,0.185538,0.964504,NaN,0.101342,0.833333,0.145720
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2623,RWA-Musanze_ADM5_83,ADM5,Kamajaga,RWA-Musanze,NaN,0.993718,0.078125,0.000000,0.000000,NaN,...,0.000000,0.006873,747.936003,-6.014612,0.321897,1.000000,0.233365,0.004286,1.000000,1.000000
2624,RWA-Musanze_ADM5_84,ADM5,Kamicaca,RWA-Musanze,NaN,0.995015,0.125000,0.000000,0.000000,NaN,...,0.000000,0.000000,753.422329,-9.194192,0.536051,1.000000,0.372657,0.047648,1.000000,0.889280
2625,RWA-Musanze_ADM5_85,ADM5,Musenyi,RWA-Musanze,NaN,0.983103,0.280000,0.000000,0.000000,NaN,...,0.000000,0.039106,754.165614,-2.157699,0.214939,1.000000,0.167535,0.000000,0.000000,1.000000
2626,RWA-Musanze_ADM5_86,ADM5,Rugari,RWA-Musanze,NaN,0.952790,0.000000,0.000000,0.000000,NaN,...,0.000000,0.000000,745.213966,-1.362025,0.488671,0.983759,0.256713,0.000000,-9999.000000,1.000000


## Upload in aws s3

In [13]:
# connect to s3
aws_credentials = pd.read_csv('/home/jovyan/PlanetaryComputerExamples/aws_credentials.csv')
aws_key = aws_credentials.iloc[0]['Access key ID']
aws_secret = aws_credentials.iloc[0]['Secret access key']

s3 = boto3.resource(
    service_name='s3',
    aws_access_key_id=aws_key,
    aws_secret_access_key=aws_secret
)

In [14]:
# upload to aws
key_data = indicators_file_aws
cities_indicators_merged.to_csv(
    f"s3://{bucket_name}/{key_data}",
    index=False,
    storage_options={
        "key": aws_key,
        "secret": aws_secret
    },
)

In [15]:
# make it public
object_acl = s3.ObjectAcl(bucket_name,key_data)
response = object_acl.put(ACL='public-read')